In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')


In [ ]:
df=pd.read_csv("C:\\Users\\Ahmed\\Desktop\\Datasets\\RTA Dataset.csv")
df.head()


In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.describe(include="all")

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df['Accident_severity'].value_counts()

In [ ]:
sns.countplot(x = df['Accident_severity'])
plt.title('Distribution of Accident severity')

In [ ]:
df.isna().sum()

In [ ]:
df.drop(['Service_year_of_vehicle','Defect_of_vehicle','Work_of_casuality', 'Fitness_of_casuality','Time'],
        axis = 1, inplace = True)
df.head()

In [ ]:
categorical=[i for i in df.columns if df[i].dtype=='O']
print('The categorical variables are',categorical)

In [ ]:
for i in categorical:
    df[i].fillna(df[i].mode()[0],inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
sns.scatterplot(x=df['Number_of_casualties'], y=df['Number_of_vehicles_involved'], hue=df['Accident_severity'])


In [ ]:
sns.jointplot(x='Number_of_casualties',y='Number_of_vehicles_involved',data=df)


In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
numerical=[i for i in df.columns if df[i].dtype!='O']
print('The numerica variables are',numerical)

In [ ]:
plt.figure(figsize=(10,10))
plotnumber = 1
for i in numerical:
    if plotnumber <= df.shape[1]:
        ax1 = plt.subplot(2,2,plotnumber)
        plt.hist(df[i],color='red')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.title('frequency of '+i, fontsize=10)
    plotnumber +=1

In [ ]:
plt.figure(figsize=(10,200))
plotnumber = 1

for col in categorical:
    if plotnumber <= df.shape[1] and col!='Pedestrian_movement':
        ax1 = plt.subplot(28,1,plotnumber)
        sns.countplot(data=df, y=col, palette='muted')
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.title(col.title(), fontsize=14)
        plt.xlabel('')
        plt.ylabel('')
    plotnumber +=1

In [ ]:
df.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

#creating a new data frame from performing the chi2 analysis
df1=pd.DataFrame()

#adding all the categorical columns except the output to new data frame
for i in categorical:
    if i!= 'Accident_severity':
        df1[i]=le.fit_transform(df[i])

In [ ]:
df1.info()

In [ ]:
plt.figure(figsize=(22,17))
sns.set(font_scale=1)
sns.heatmap(df1.corr(), annot=True)

In [ ]:
df1.head()

In [ ]:
from sklearn.feature_selection import chi2
f_p_values=chi2(df1,df['Accident_severity'])

In [ ]:
f_p_values

In [ ]:
f_p_values1=pd.DataFrame({'features':df1.columns, 'Fscore': f_p_values[0], 'Pvalues':f_p_values[1]})
f_p_values1


In [ ]:
f_p_values1.sort_values(by='Pvalues',ascending=True)

In [ ]:
df2=df.drop(['Owner_of_vehicle', 'Type_of_vehicle', 'Road_surface_conditions', 'Pedestrian_movement',
         'Casualty_severity','Educational_level','Day_of_week','Sex_of_driver','Road_allignment',
         'Sex_of_casualty'],axis=1)
df2.head()

In [ ]:
df2.shape

In [ ]:
df2.info()

In [ ]:
categorical_new=[i for i in df2.columns if df2[i].dtype=='O']
print(categorical_new)

In [ ]:
for i in categorical_new:
    print(df2[i].value_counts())

In [ ]:
dummy=pd.get_dummies(df2[['Age_band_of_driver', 'Vehicle_driver_relation', 'Driving_experience',
                          'Area_accident_occured', 'Lanes_or_Medians', 'Types_of_Junction', 'Road_surface_type', 
                          'Light_conditions', 'Weather_conditions', 'Type_of_collision', 'Vehicle_movement', 
                          'Casualty_class', 'Age_band_of_casualty', 'Cause_of_accident']],drop_first=True)
dummy.head()

In [ ]:
x=df3.drop(['Accident_severity'],axis=1)
x.shape

In [ ]:
x.head()

In [ ]:
y=df3.iloc[:,2]
y.head()

In [ ]:
y.value_counts()

In [ ]:
sns.countplot(x = y, palette='muted')

In [ ]:
from imblearn.over_sampling import SMOTE
oversample=SMOTE()
xo,yo=oversample.fit_resample(x,y)

In [ ]:
y1=pd.DataFrame(yo)
y1.value_counts()

In [ ]:
sns.countplot(x = yo, palette='muted')

In [ ]:
from sklearn.model_selection import train_test_split
#splitting 70% of the data to training data and 30% of data to testing data
x_train,x_test,y_train,y_test=train_test_split(xo,yo,test_size=0.30,random_state=42)




In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_KNN=KNeighborsClassifier(n_neighbors=5)
model_KNN.fit(x_train,y_train)

In [ ]:
y_pred=model_KNN.predict(x_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,ConfusionMatrixDisplay

In [ ]:
report_KNN=classification_report(y_test,y_pred)
print(report_KNN)

In [ ]:
accuracy_KNN=accuracy_score(y_test,y_pred)
print(accuracy_KNN)

In [ ]:
matrix_KNN=confusion_matrix(y_test,y_pred)
print(matrix_KNN,'\n')
print(ConfusionMatrixDisplay.from_predictions(y_test,y_pred))
s